In [ ]:
import xgboost
import shap
import matplotlib.pylab as pl
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os
from sklearn.metrics import roc_auc_score, average_precision_score
import argparse
from feature_selector import FeatureSelector
from sklearn.model_selection import GridSearchCV
import pickle
import sklearn
import scipy as sp
from scipy.cluster.hierarchy import complete, fcluster
from collections import defaultdict

In [ ]:
def label(permth, mortstat, month):
    if permth > month:
        return 0
    else:
        if mortstat == 1:
            return 1
        else:
            return 2

In [ ]:
def bootstrap_ci(y_pre, y_label, sample_size, repetitions = 1000, alpha = 0.05): 
    y_pre = np.array(y_pre)
    y_label = np.array(y_label)
    
    auc = []
    ap = []
    for i in range(repetitions):
        np.random.seed(i)
        idx = list(np.random.choice(len(y_pre), replace = True, size = sample_size))
        y_pre_bootstrap = y_pre[idx]
        y_label_bootstrap = y_label[idx]
        auc.append(roc_auc_score(y_label_bootstrap, y_pre_bootstrap))
        ap.append(average_precision_score(y_label_bootstrap, y_pre_bootstrap))
    # confidence interval
    left_auc = np.percentile(auc, alpha/2*100)
    right_auc = np.percentile(auc, 100-alpha/2*100)
    left_ap = np.percentile(ap, alpha/2*100)
    right_ap = np.percentile(ap, 100-alpha/2*100)
    # point estimate
    print('average AUROC', np.mean(auc))
    print((1-alpha)*100,'%','confidence interval for the AUROC:', (round(left_auc,4), round(right_auc,4)))
    print('average AP', np.mean(ap))
    print((1-alpha)*100,'%','confidence interval for the AP:', (round(left_ap,4), round(right_ap,4)))
    return auc, left_auc, right_auc, ap, left_ap, right_ap

In [ ]:
year_num = 5
path = './model/supervised_distance_feature_elimination/'
if not os.path.isdir(path):
    os.mkdir(path)

In [ ]:
X = pd.read_csv('./data/NHANES/NHANES.csv')
if str(year_num)+'_year_label' not in X.columns:
    X[str(year_num)+'_year_label'] = X.apply(lambda x: label(x['permth_int'], x['mortstat'], 12*int(year_num)), axis=1)
    
X = X[X[str(year_num)+'_year_label']!=2]
y = X[str(year_num)+'_year_label']

if int(year_num) not in [1,2,3,4,5]:
    X = X.drop([str(year_num)+'_year_label'], axis=1)

mortstat = X['mortstat']
permth_int = X['permth_int']
drop_list = ["mortstat", "permth_int", '1_year_label', '2_year_label', '3_year_label', '4_year_label', '5_year_label']
X = X.drop(drop_list, axis=1)

X = X.drop(['Demographics_ReleaseCycle'], axis=1)

fea_list = pd.read_csv('./data/NHANES/NHANES_feature_list.csv')
nominal_fea = fea_list[fea_list['Nominal']==1]['Type_Short_Name'].tolist()
nominal_fea = list(set(nominal_fea) & set(X.columns))
X = pd.get_dummies(X, columns=nominal_fea, drop_first=True)

display_name = pd.read_csv('./data/NHANES/NHANES_feature_list_Display_name.csv')
display_col=[]
for col in X.columns:
    display_col.append(list(display_name.loc[display_name['Type_Short_Name']==col, 'Display_Name'])[0])
col_dict = dict(zip(X.columns, display_col))

print(X.columns)
print('After encoding', X.shape)

In [ ]:
print(X.columns)
print(X.shape)
print('# samples: ', X.shape[0])
print('# positive samples: ', sum(y==1))
print('# negative samples: ', sum(y==0))
print('# features: ', X.shape[1])  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=7)

y_train = np.array(y_train); y_test = np.array(y_test); y_val = np.array(y_val)

In [ ]:
clustering = shap.utils.hclust(X_train, y_train, random_state=7)

In [ ]:
maxclust_list = [151] + [i for i in range(145, 50, -5)] + [53, 1]
X_train_all = X_train.copy()
X_test_all = X_test.copy()
X_val_all = X_val.copy()

features_ranking_dict = {}
auc_dict = {}
ap_dict = {}
left_auc_dict = {}
right_auc_dict = {}
left_ap_dict = {}
right_ap_dict = {}

In [ ]:
features_input = {}
ranked_features = list(X_train_all.columns)
feature_temp = ranked_features
for maxclust in maxclust_list:
    print('maxclust: ', maxclust)
    if maxclust == X_train_all.shape[1]:
        print('# features: ', maxclust)
        X_train = X_train_all
        X_test = X_test_all
        X_val = X_val_all
        cluster_num = maxclust
        features_input[cluster_num] = feature_temp
    else:
        cluster_temp = fcluster(clustering, maxclust, criterion='maxclust')
        cluster_num = max(cluster_temp)
        if (max(cluster_temp) == len(ranked_features)):
            features_input[cluster_num] = feature_temp
            continue
        cluster_dict = defaultdict(list)
        cluster_rank_dict = defaultdict(list)
        for fea, cluster_id in zip(X_train_all.columns, cluster_temp):
            if fea in ranked_features:
                cluster_dict[cluster_id].append(fea)
                cluster_rank_dict[cluster_id].append(ranked_features.index(fea))
        feature_temp = []
        for cluster_id in cluster_dict:
            feature_temp.append(cluster_dict[cluster_id][np.argmin(cluster_rank_dict[cluster_id])])
        print('# features: ', len(feature_temp))
        X_train = X_train_all.loc[:, feature_temp]
        X_test = X_test_all.loc[:, feature_temp]
        X_val = X_val_all.loc[:, feature_temp]
        features_input[cluster_num] = feature_temp
    xlf = xgboost.XGBClassifier(n_estimators=1000, max_depth=4, subsample=0.5, min_child_weight=3, objective='binary:logistic', random_state=7)
    xlf.fit(X_train, y_train, eval_set = [(X_val, y_val)], early_stopping_rounds=100, verbose=False)
    model_train = xlf
    pickle.dump(model_train, open(path+"model_"+str(cluster_num)+".pickle.dat", "wb"))
    y_pre = model_train.predict_proba(X_test)[:, 1]
    auc, left_auc, right_auc, ap, left_ap, right_ap = bootstrap_ci(y_pre, y_test, len(y_test), repetitions = 1000, alpha = 0.05)
    auc_dict[cluster_num] = auc
    ap_dict[cluster_num] = ap
    left_auc_dict[cluster_num] = left_auc
    right_auc_dict[cluster_num] = right_auc
    left_ap_dict[cluster_num] = left_ap
    right_ap_dict[cluster_num] = right_ap    


    if len(X_train)>=5000:
        back_data = X_train.sample(n=5000, random_state=428)
    else:
        back_data = X_train
    if len(X_test)>=2000:
        fore_data = X_test.sample(n=2000, random_state=528)
        fore_data_label = pd.DataFrame(y_test).sample(n=20, random_state=528)
    else:
        fore_data = X_test
        fore_data_label = pd.DataFrame(y_test)

    explainer = shap.TreeExplainer(model_train, data=back_data)
    shap_values = explainer.shap_values(fore_data)
    ranked_features = list(X_train.columns[np.argsort(-np.sum(np.abs(shap_values), axis=0))])

    features_ranking_dict[cluster_num] = ranked_features

In [ ]:
pickle.dump(auc_dict, open(path+"auc_dict.pickle.dat", "wb"))
pickle.dump(ap_dict, open(path+"ap_dict.pickle.dat", "wb"))
pickle.dump(left_auc_dict, open(path+"left_auc_dict.pickle.dat", "wb"))
pickle.dump(right_auc_dict, open(path+"right_auc_dict.pickle.dat", "wb"))
pickle.dump(left_ap_dict, open(path+"left_ap_dict.pickle.dat", "wb"))
pickle.dump(right_ap_dict, open(path+"right_ap_dict.pickle.dat", "wb"))
pickle.dump(features_ranking_dict, open(path+"features_ranking_dict.pickle.dat", "wb"))
pickle.dump(features_input, open(path+"features_input.pickle.dat", "wb"))